5.7 오차역전파법 구현

5.7.1 신경망 학습의 overview
    A. 전제: 신경망에 적용 가능한 가중치와 편향 존재, 이 수치를 조정하는 과정을 '학습'이라고 지칭

    B. 미니배치: 훈련 데이터 중 일부 추출, 미니배치의 손실 함수 값을 줄이는 것을 목표로 함
    C. 기울기 산출: 각 가중치 매개변수의 기울기 연산-> 손실 함수의 값을 가장 작게 하는 방향 제시
    D. 매개변수 갱신: 기울기 방향으로 '아주 조금' 갱신
    E. 반복: B~E 단계 반복

    - 이 중, 오차역전파법은 C. 기울기 산출에서 등장
    - 수치 미분 사용: 구현 용이, but 긴 계산 시간
    - or 오차역전파법: 효율적으로 연산 가능

5.7.2 오차역전파법을 적용한 신경망 구현
    - 2층 신경망 구현

    - 변수
        A. params: 딕셔너리, 신경망의 매개변수(W: 가중치, b: 편향)
        B. layers: 순서가 있는 딕셔너리, 신경망의 계층(Affine)
        C. lastLayer: 신경망의 마지막 계층, 예시에서는 SoftmaxWithLoss 계층 활용

    - 메서드
        A. __init__: 초기화, 인수(입력층 뉴런 수, 은닉층 뉴런 수, 출력층 뉴런 수, 가중치 초기화 시 정규분포의 스케일)
        B. predict(self.x): 예측(추론) 수행, x: 이미지 데이터
        C. loss(Self.x, t): 손실함수 값 연산, t: 정답 레이블
        D. accuracy(self.x, t): 정확도
        E. numerical_gradient(self.x, t): 가중치 매개변수의 기울기를 수치 미분 방식으로 연산
        F. gradient(self.x, t): 가중치 매개변수의 기울기를 오차역전파법으로 연산

In [5]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None        # 손실
        self.y= None            # softmax의 출력
        self.t = None           # 정답 레이블(원-핫 벡터)

    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)

        return self.loss

    # 역전파의 경우, 전파하는 값을 배치의 수로 나누어 데이터 1개당 오차를 앞 계층으로 전파
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size

        return dx

In [12]:
import sys, os
sys.path.append(os.pardir)
import numpy as np

from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict             # 순서가 있는 딕셔너리

class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)

        #계층 생성
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])

        self.lasyLayer = SoftmaxWithLoss()

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)

        return x

        # return 뒤에 이렇게 두면 인식이 되나여,,? 교재 오타인감,,
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])

        self.lastLayer = SoftmaxWithLoss()

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)            # 순전파 때, 추가한 순서대로 각 계층의 forward 메서드 호출

        return x

    # x: 입력 데이터, t: 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        return self.lasyLayer.forward(y, t)

    def accuracy(self, x, t):
        y = self.predict(X)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)

        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy

    # x: 입력 데이터, t: 정답 레이블
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)

        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])

        return grads

    def gradient(self, x, t):
        # 순전파
        self.loss(x, t)

        # 역전파
        dout = 1
        dout = self.lasyLayer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)         # 역전파도 계층 역순으로 호출

        # 결과 저장
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db

        return grads


5.7.3 오차역전파법으로 구한 기울기 검증
    - 매개변수가 많아도 효율적으로 연산할 수 있음
    - but, 수치 미분도 오차역전파법을 정확히 구현했는지 확인하기 위해 필요
    == 기울기 확인(gradient check)

In [16]:
from locale import normalize
from operator import ne
import sys, os
sys.path.append(os.pardir)
import numpy as np
from mnist import load_mnist
# from two_layer_net import TwoLayerNet

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)


# 각 가중치의 절대 오차의 평균 연산
for key in grad_numerical.keys():
    diff = np.average(np.abs(grad_backprop[key] - grad_numerical[key]))
    print(key + ":" + str(diff))

- 수치 미분과 오차역전파법의 결과 오차가 0이 되는 경우 드뭄
== 계산의 정밀도가 유한하기 때문(32bit 부동소수점과 같이)

- 오차가 큰 경우, 오차역전파법 구현을 다시 체크해야